In [2]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [5]:
# hyper parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# image processing
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5),
                        std=(0.5, 0.5, 0.5))])
mnist = torchvision.datasets.MNIST(root = 'data/',
                                  train = True,
                                  transform = transform,
                                  download = True)
data_loader = torch.utils.data.DataLoader(dataset = mnist,
                                         batch_size = batch_size,
                                         shuffle = True)

In [6]:
# discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2), #0.2: Controls the angle of the negative slope
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# generator
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# device setting
D = D.to(device)
G = G.to(device)

# loss and optimizer
criterion = nn.BCELoss() # binary cross entropy loss
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

In [8]:
def denorm(x):
    out = (x+1)/2
    return out.clamp(0, 1) #min=0, max=1

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

In [ ]:
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, 1).to(device)
        
        # create the labels that are later used as input for BCE loss
        real_labels = torch.ones(batch_size, 1).to(device) # the 1s
        fake_labels = torch.zeros(batch_size, 0).to(device) # the 0s
        
        # train discriminator
        